*Note:  This notebooks should be executed against a Serverelss SQL Warehouse.*

# Creating the FHIR Bundle Meta Table
***

## Notebook Setup
***

In [0]:
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
USE IDENTIFIER("fhir_workshop." || schema_use);

In [0]:
SELECT current_catalog(), current_schema();

## FHIR Bundle Meta 
***

Depending on the source of the FHIR bundle there may be a full set of metadata about the bundle itself.  This is standard practice for providers that clean and standardize your incoming FHIR data, or convert your C-CDAs, ADTs, or other clinical data to the FHIR format such as Redox.  Note that Synthea doesn't include a bundle meta, therefore we're not going to fully parse the Meta object quite as much as we would if it were available.  We do have one Redox JSON loaded into our bronze tables that we can at least review.  Normally we'd split the Meta object object into separate columns exactly like we would for the Bundle's "entry resources" but that would be overkill for this class. 

What is helpful however is to have a place for items such as the `file_metadata` and then `ingest_time` avaialble in a silver table so that we don't need to continue to bring that along for the ride with the Resource tables.  

In [0]:
DROP TABLE IF EXISTS bundle_meta;

In [0]:
CREATE OR REFRESH STREAMING TABLE bundle_meta (
  bundle_uuid STRING PRIMARY KEY COMMENT 'Unique identifier for the FHIR bundle and the primary key for the table.  This column will be required for all joins amongst resource tables as Resource primary keys are only guaranteed to be unique inside of a bundle.'
  ,file_metadata STRUCT<
    file_path: STRING
    ,file_name: STRING
    ,file_size: BIGINT
    ,file_block_start: BIGINT
    ,file_block_length: BIGINT
    ,file_modification_time: TIMESTAMP
  > COMMENT 'Original meta date of the file ingested from the volume.'
  ,ingest_time TIMESTAMP COMMENT 'The date timestamp the file was ingested.'
  ,bundle_resourceType STRING COMMENT "The FHIR Bundle's overall resource type."
  ,bundle_type STRING COMMENT "The FHIR Bundle's overall type."
  ,meta VARIANT COMMENT 'Metadata about the FHIR Bundle overall, typically includes information about the source of the bundle, but is not required to be passed for a valid HL7 transaction.  Always NULL for data that originated from Synthea.'
)
COMMENT 'Original FHIR Bundle Metadata'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  bundle_uuid
  ,file_metadata
  ,ingest_time
  ,fhir:resourceType::string as bundle_resourceType
  ,fhir:type::string as bundle_type
  ,fhir:Meta as meta
FROM 
  STREAM (fhir_bronze_variant);

In [0]:
SHOW CREATE TABLE bundle_meta;

In [0]:
SELECT 
 *
FROM 
  bundle_meta
WHERE 
  file_metadata.file_name = 'DBPriorAuthExample.json'